In [1]:
import numpy as np
import pandas as pd

# Data Viz. 
import statsmodels.formula.api as smf
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.ndimage import gaussian_filter
from calendar import monthrange
from calendar import month_name

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from matplotlib import rcParams
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
RSEED=42
import datetime, time, os

In [11]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [3]:
df = pd.read_csv('data/train.csv')
df

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,20,50,0.000000,0.083334,0,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.907794
2,3,1,20,50,0.067514,22.509278,0,7.876254
3,4,1,20,50,0.101542,22.808822,0,11.742872
4,5,1,20,50,0.135756,25.355850,0,12.234987
...,...,...,...,...,...,...,...,...
6035995,6035996,125749,50,10,2.504603,1.489714,1,3.869032
6035996,6035997,125749,50,10,2.537961,1.488497,1,3.869032
6035997,6035998,125749,50,10,2.571408,1.558978,1,3.798729
6035998,6035999,125749,50,10,2.604744,1.272663,1,4.079938


In [6]:
df['vol'] = df['time_step'] * df['u_in']
df['vol'] = df.query('u_out==0').groupby('breath_id')['vol'].cumsum()
df['rtime']=df['time_step'].apply(lambda x: round(x,3))
df.query('id%80==2')
df['step_id']=df.id.apply(lambda x: x%80)
df=df.query('u_out==0')
df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
df['minus_one']=-1.0
df['plus_one']=1.0
df['exponent']=(df['minus_one']*df['time_step'])/(df['R']*df['C'])
df['factor']=np.exp(df['exponent'])
df['vf']=(df['u_in_cumsum']*df['R'])/df['factor']
df['vt']=0
df.loc[df['time_step'] != 0, 'vt']=df['vol']/(df['C']*(df['minus_one']*df['factor']+df['plus_one']))
df['v']=df['vf']+df['vt']

/var/folders/d5/ywbj67kx54n4fq9j7nf3v4wh0000gn/T/ipykernel_48764/1413260000.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vol'] = df['time_step'] * df['u_in']
/var/folders/d5/ywbj67kx54n4fq9j7nf3v4wh0000gn/T/ipykernel_48764/1413260000.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vol'] = df.query('u_out==0').groupby('breath_id')['vol'].cumsum()
/var/folders/d5/ywbj67kx54n4fq9j7nf3v4wh0000gn/T/ipykernel_48764/1413260000.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [8]:
cols=['v','u_in_cumsum','vol','pressure', 'u_in', 'R', 'C', 'time_step', 'step_id']
red_df=df[cols]
red_df

,v,u_in_cumsum,vol,pressure,u_in,R,C,time_step,step_id
0,1.666680,0.083334,0.000000,5.837492,0.083334,20,50,0.000000,1
1,737.006954,18.466375,0.618632,5.907794,18.383041,20,50,0.033652,2
2,1453.034477,40.975653,2.138333,7.876254,22.509278,20,50,0.067514,3
3,2153.211108,63.784476,4.454391,11.742872,22.808822,20,50,0.101542,4
4,2946.481672,89.140326,7.896588,12.234987,25.355850,20,50,0.135756,5
...,...,...,...,...,...,...,...,...,...
6035945,15962.475671,238.890288,66.643100,29.459013,1.869367,50,10,0.834147,26
6035946,16025.086384,241.044703,68.512214,29.107502,2.154414,50,10,0.867574,27
6035947,16010.376008,242.349137,69.687402,29.880825,1.304434,50,10,0.900917,28
6035948,16046.588564,244.082966,71.307334,29.388710,1.733830,50,10,0.934309,29


In [9]:
#train test split
target="pressure"
X = red_df.drop(target, axis=1)
y = red_df.loc[:,target].values
#y = df_dmy[target]
#y = np.array(df.pop("target"))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state=RSEED)#, stratify=False)
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)


scaler= StandardScaler()
#num_list = list(X_train.columns[X_train.dtypes!=object])
col_names = ['v', 'u_in_cumsum', 'vol', 'u_in',	'R', 'C']
X_train_num=X_train[col_names].copy()

X_train[col_names] = scaler.fit_transform(X_train[col_names])
X_test[col_names] = scaler.transform(X_test[col_names])

X_train shape: (1603677, 8)
y_train shape: (1603677,)
X_test shape: (687291, 8)
y_test shape: (687291,)


In [12]:
# With this command you can clear any logs from previous runs
# If you want to compare different runs you can skip this cell 
!rm -rf my_logs/

In [13]:
# Define path for new directory 
root_logdir = os.path.join(os.curdir, "my_logs_second")
root_logdir

'./my_logs_second'

In [14]:
# Define function for creating a new folder for each run
def get_run_logdir():
    run_id = time.strftime('run_%d_%m_%Y-%H_%M_%S')
    return os.path.join(root_logdir, run_id)

In [15]:
run_logdir = get_run_logdir()
run_logdir

'./my_logs_second/run_21_04_2022-17_39_34'

In [16]:
# Create function for using callbacks; "name" should be the name of the model you use
def get_callbacks(name):
    return tf.keras.callbacks.TensorBoard(run_logdir+name, histogram_freq=1)

In [18]:
#choosing the variable and normalizing it
#col_var = ['v', 'u_in_cumsum', 'vol', 'u_in', 'R', 'C']
new_var = np.array(X_train)

new_all_normalizer = preprocessing.Normalization(input_shape = [8,], axis = None)
new_all_normalizer.adapt(new_var)

2022-04-21 17:56:28.319502: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-21 17:56:28.343518: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [19]:
#Build a model
def build_and_compile_model7(norm):
    model = keras.Sequential([
        norm,
        layers.Dense(80, activation='selu'),
        layers.Dense(80, activation='selu'),
        layers.Dense(80, activation='selu'),
        layers.Dense(80, activation='selu'),
        layers.Dense(80, activation='selu'),
        layers.Dense(1)
    ])

    model.compile(loss='mae',
                  metrics='mae',
                  optimizer=tf.keras.optimizers.Adam(0.01))
    return model

In [20]:
#building the model using single input
dnn_new_all_model7 = build_and_compile_model7(new_all_normalizer)

In [21]:
#model summary
dnn_new_all_model7.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normalizat  (None, 8)                3         
 ion)                                                            
                                                                 
 dense (Dense)               (None, 80)                720       
                                                                 
 dense_1 (Dense)             (None, 80)                6480      
                                                                 
 dense_2 (Dense)             (None, 80)                6480      
                                                                 
 dense_3 (Dense)             (None, 80)                6480      
                                                                 
 dense_4 (Dense)             (None, 80)                6480      
                                                        

In [22]:
#Train the model
#%%time
history = dnn_new_all_model7.fit(
    X_train, y_train,
    validation_split=0.2,
    verbose=1, epochs=500,
    batch_size = 500,
    callbacks=get_callbacks("dnn_all_var_large_5layer_ep500_selu"))

Epoch 1/500


2022-04-21 18:01:06.150926: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2566/2566 [==============================] - ETA: 0s - loss: 2.1924 - mae: 2.1924

2022-04-21 18:01:52.442205: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2566/2566 [==============================] - 53s 19ms/step - loss: 2.1924 - mae: 2.1924 - val_loss: 1.7577 - val_mae: 1.7577
Epoch 2/500
2566/2566 [==============================] - 48s 19ms/step - loss: 1.7019 - mae: 1.7019 - val_loss: 1.6101 - val_mae: 1.6101
Epoch 3/500
2566/2566 [==============================] - 47s 18ms/step - loss: 1.5950 - mae: 1.5950 - val_loss: 1.6085 - val_mae: 1.6085
Epoch 4/500
2566/2566 [==============================] - 49s 19ms/step - loss: 1.5587 - mae: 1.5587 - val_loss: 1.6265 - val_mae: 1.6265
Epoch 5/500
2566/2566 [==============================] - 48s 19ms/step - loss: 1.5292 - mae: 1.5292 - val_loss: 1.5209 - val_mae: 1.5209
Epoch 6/500
2566/2566 [==============================] - 48s 19ms/step - loss: 1.4934 - mae: 1.4934 - val_loss: 1.4656 - val_mae: 1.4656
Epoch 7/500
2566/2566 [==============================] - 50s 20ms/step - loss: 1.4723 - mae: 1.4723 - val_loss: 1.4819 - val_mae: 1.4819
Epoch 8/500
2566/2566 [==============================